# Boundary Equilibrimum BiGANs add Energy Loss example

## 초기 설정들

In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import time
import time 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 

os.environ["CUDA_VISIBLE_DEVICES"]="1"


import pickle
import gzip

file_dir = 'anoGANs_MNIST_data/'

with gzip.open(file_dir + 'test_normal_data.pickle.gzip','rb') as f :
    test_normal_data = pickle.load(f)
    print('test_normal_data : ' ,test_normal_data.shape)

with gzip.open(file_dir + 'test_anomalous_data.pickle.gzip','rb') as f :
    test_anomalous_data = pickle.load(f)
    print('test_anomalous_data : ',test_anomalous_data.shape)
    
with gzip.open(file_dir + 'train_normal_data.pickle.gzip','rb') as f :
    train_normal_data = pickle.load(f)
    print('train_normal_data : ', train_normal_data.shape)
    
with gzip.open(file_dir + 'train_anomalous_data.pickle.gzip','rb') as f :
    train_anomalous_data = pickle.load(f)
    print('train_anomalous_data : ',train_anomalous_data.shape )

def idx_shuffle(x) : 
    l = x.shape[0]
    idx = np.arange(l)
    np.random.shuffle(idx)
    shuffled_x = np.empty(x.shape)

    for i in range(l):
        shuffled_x[idx[i]] = x[i]
    
    return shuffled_x

def mnist_4by4_save(samples,path):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)    
    gs.update(wspace=0.05, hspace=0.05) #이미지 사이간격 조절
  
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')    
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
   
        plt.imshow(sample.reshape(64, 64), cmap='Greys_r',clim=(0.0,1.0))
    plt.colorbar()
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)
   
    return None

def gan_loss_graph_save(G_loss,D_loss,path):
    x1 = range(len(G_loss))
    x2 = range(len(D_loss))
      
    y1 = G_loss
    y2 = D_loss
  
      
    plt.plot(x1,y1,label='G_loss') 
    plt.plot(x2,y2,label='D_loss') 
  
    plt.xlabel('weight per update')
    plt.ylabel('loss')             
    plt.legend(loc=4)              
    plt.grid(True)
    plt.tight_layout()
  
    plt.savefig(path)              

    return None

file_name = 'ex_BE_BiGANs_add_Energy_loss'

if not os.path.isdir(file_name) :
    os.mkdir(file_name)


test_normal_data :  (5139, 64, 64, 1)
test_anomalous_data :  (4861, 64, 64, 1)
train_normal_data :  (28038, 64, 64, 1)
train_anomalous_data :  (26962, 64, 64, 1)


## 네트워크 정의

D부분을 encoder와 discriminator로 나눈 이유는 encoder를 나중에 feature map으로 쓰기 위해서 편의상 나누어서 정의함



In [2]:

def G(x,isTrain = True, reuse = False, name = 'G') : #input = (minibatch * w * h * ch)
    
    # out size = (in size + 2*padding - kenel)/strides + 1    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)

    with tf.variable_scope('G',reuse=reuse)  :
        
        #x = (-1, 1, 1, 100)

        conv1 = tf.layers.conv2d_transpose(x,512,[4,4], strides=(1,1),padding = 'valid',
                kernel_initializer=w_init, bias_initializer=b_init) 
        r1 = tf.nn.elu(tf.layers.batch_normalization(conv1,training=isTrain))#4*4*512
        
        conv2 = tf.layers.conv2d_transpose(r1,256,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#8*8*256
                
        conv3 = tf.layers.conv2d_transpose(r2,128,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#16*16*128

        conv4 = tf.layers.conv2d_transpose(r3,64,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain))#32*32*64

        conv5 = tf.layers.conv2d_transpose(r4,1,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init) #64*64*1
        
    r5= tf.nn.tanh(conv5,name=name)#64*64*1
  
    return r5

def E(x,isTrain = True, reuse = False, name = 'E') : #input = (minibatch * w * h * ch)
    
    # out size = (in size + 2*padding - kenel)/strides + 1    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)

    with tf.variable_scope('E',reuse=reuse)  :
        
        #x = (-1, 64, 64, 1)

        conv1 = tf.layers.conv2d(x,64,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init) 
        r1 = tf.nn.elu(conv1)#32*32*64
        
        conv2 = tf.layers.conv2d(r1,128,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#16*16*128
                
        conv3 = tf.layers.conv2d(r2,256,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#8*8*256

        conv4 = tf.layers.conv2d(r3,512,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain))#4*4*512

        conv5 = tf.layers.conv2d(r4,100,[4,4], strides=(1,1),padding = 'valid',
                kernel_initializer=w_init, bias_initializer=b_init) #1*1*100
        
    r5= tf.add(conv5, 0 ,name=name)#1*1*100
  
    return r5

def D_enc(x,z,isTrain=True,reuse = False, name = 'D_enc') :
    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    with tf.variable_scope('D_enc', reuse=reuse) :
        
        #x = (-1,64,64,1)
        # out size = (in size + 2*padding - kenel)/strides + 1   
        z_flat = tf.reshape(z, (-1, z_size))
        
        w1 = tf.get_variable('w1',[z_flat.get_shape()[1], 64*64*2],initializer=w_init)
        b1 = tf.get_variable('b1',[64*64*2],initializer=b_init)
        fc1 = tf.nn.elu(tf.matmul(z_flat,w1) + b1)
          
        r0 = tf.reshape(fc1, (-1, 64,64,2)) # 64*64*2
        
        x_cat = tf.concat([x, r0],3)
        
        conv1 = tf.layers.conv2d(x_cat,64,[5,5], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init) 
        r1 = tf.nn.elu(conv1)#32*32*64

   
        conv2 = tf.layers.conv2d(r1,128,[5,5], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#16*16*128

  
        conv3 = tf.layers.conv2d(r2,256,[5,5], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#8*8*256
        
        conv4 = tf.layers.conv2d(r3,512,[5,5], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)    
        r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain), name = name)#4*4*512
        
        conv5 = tf.layers.conv2d(r4,100,[4,4], strides=(1,1),padding = 'valid',
                                kernel_initializer=w_init, bias_initializer=b_init)    
        r5 = tf.layers.batch_normalization(conv5,training=isTrain)
    return tf.add(r5,0,name=name)

def D_dec(x,isTrain=True,reuse = False, name = 'D_dec') :
    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    with tf.variable_scope('D_dec', reuse=reuse) :
        
        #x = (-1,64,64,1)
        # out size = (in size + 2*padding - kenel)/strides + 1   
     
        conv6 = tf.layers.conv2d_transpose(x,512,[4,4], strides=(1,1),padding = 'valid',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r6 = tf.nn.elu(tf.layers.batch_normalization(conv6,training=isTrain))#4*4*256
        
        conv7 = tf.layers.conv2d_transpose(r6,256,[5,5], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r7 = tf.nn.elu(tf.layers.batch_normalization(conv7,training=isTrain))#8*8*256


        conv8 = tf.layers.conv2d_transpose(r7,128,[5,5], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r8 = tf.nn.elu(tf.layers.batch_normalization(conv8,training=isTrain))#16*16*128
             
        conv9 = tf.layers.conv2d_transpose(r8,64,[5,5], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r9 = tf.nn.elu(tf.layers.batch_normalization(conv9,training=isTrain))#32*32*64
          
        conv10 = tf.layers.conv2d_transpose(r9,1,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init) #64*64*1
        
    r10= tf.nn.tanh(conv10,name=name)#64*64*1
    
    return r10

## 네트워크 연결

In [3]:

train_epoch = 100
batch_size = 100
z_size = 100
m = 10
lam = 0.01
gamma = 0.7
k_curr = 0.001

z = tf.placeholder(tf.float32,shape=(None,1,1,z_size),name = 'z')    #x_z = G(z)
u = tf.placeholder(tf.float32, shape = (None, 64,64,1),name='u')      #u = x
k = tf.placeholder(tf.float32, name = 'k')


isTrain = tf.placeholder(dtype=tf.bool,name='isTrain')  # BN 설정 parameter


G_sample = G(z,isTrain,name='G_sample') # G(z)
E_z = E(u,isTrain,name = 'E_z') 

re_image = G(E_z, isTrain, reuse=True, name ='re_image')


D_real = D_dec(D_enc(u, E_z, isTrain,reuse=False), isTrain, reuse=False, name = 'D_real')                       # D(x)
D_fake = D_dec(D_enc(G_sample,z, isTrain,reuse=True), isTrain, reuse=True, name = 'D_fake')         # D(G(z))

#input = (minibatch * w * h * ch)
D_real_loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum((D_real-u)**2, axis=[1,2,3])) , name = 'D_real_loss')                         

D_fake_loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum((D_fake - G_sample)**2, axis=[1,2,3])),  name = 'D_fake_loss' )

D_loss =  tf.add(D_real_loss, -k*D_fake_loss, name='D_loss')                                      

G_loss =  tf.reduce_mean(tf.sqrt(tf.reduce_sum((D_fake - G_sample)**2, axis=[1,2,3])), name='G_loss')                             # E[-log(D(G(z)))]

temp1 = tf.maximum(0.0, m - tf.sqrt(tf.reduce_sum((D_real - u)**2, axis=[1,2,3])))
E_loss = tf.reduce_mean(temp1,  name = 'E_loss' )




T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('D_dec') or var.name.startswith('D_enc')]
G_vars = [var for var in T_vars if var.name.startswith('G')]
E_vars = [var for var in T_vars if var.name.startswith('E')]

    # When using the batchnormalization layers,
    # it is necessary to manually add the update operations
    # because the moving averages are not included in the graph
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)) :        
    D_optim = tf.train.AdamOptimizer(2e-5,beta1=0.5).minimize(D_loss, var_list=D_vars, name='D_optim') 
    G_optim = tf.train.AdamOptimizer(2e-4,beta1=0.5).minimize(G_loss, var_list=G_vars, name='G_optim')
    E_optim = tf.train.AdamOptimizer(2e-4,beta1=0.5).minimize(E_loss, var_list=E_vars, name='E_optim')



## 학습

In [ ]:
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess :
    sess.run(tf.global_variables_initializer())

    np.random.seed(int(time.time()))
    tf.set_random_seed(int(time.time()))
    
    mnist_4by4_save(np.reshape(test_normal_data[0:16],(-1,64,64,1)),file_name + '/D_origin.png')    
    test_z = np.random.uniform(-1,1,size=(16,1,1,z_size))

    log_txt = open(file_name +'/log.txt','w')

    hist_G = []
    hist_D = []
    G_error = []
    D_error = []
    E_error = []
    D_fake_error = []
    D_real_error = []
    new_measure = []
    new_k = []
    
    start = time.time()
    for epoch in range(train_epoch) :
        
        train_normal_data = idx_shuffle(train_normal_data) 
        
        for iteration in range(train_normal_data.shape[0] // batch_size) : 
        
            
            train_images = train_normal_data[iteration*batch_size : (iteration+1)*batch_size]      
            u_ = np.reshape(train_images,(-1,64,64,1)) 
            z_ = np.random.uniform(-1,1,size=(batch_size,1,1,z_size))
            
        
            _ , D_e,D_real_e,D_fake_e = sess.run([D_optim, D_loss,D_real_loss,D_fake_loss], {u : u_, z : z_, k : k_curr, isTrain : True})
            D_error.append(D_e)
            D_real_error.append(np.maximum(0.0, D_real_e))
            D_fake_error.append(np.maximum(0.0,D_fake_e))

            #    train_images,train_labels = mnist.train.next_batch(100)    
            #    u_ = np.reshape(train_images,(-1,64,64,1)) 
            #    z_ = np.random.normal(0,1,size=(100,1,1,100))
   
            _ , G_e = sess.run([G_optim, G_loss], {u : u_, z : z_, k : k_curr,  isTrain : True}) 
            G_error.append(G_e)
            _ , E_e = sess.run([E_optim, E_loss], {u : u_, z : z_,  k : k_curr, isTrain : True}) 
            E_error.append(E_e)
            
            k_curr = k_curr + lam * (gamma*D_real_e - G_e)
            
            measure = D_real_e + np.abs(gamma*D_real_e - G_e)
            
            new_measure.append(measure)
            new_k.append(k_curr)

        hist_D.append(np.mean(D_error)) 
        hist_G.append(np.mean(G_error))

        print('D_e : %.3f, D_real_e : %.3f, D_fake_e : %.3f, G_e : %.3f, E_e : %.3f, new_measure : %.3f, k_curr : %3f'%(np.mean(D_error), np.mean(D_real_error),
            np.mean(D_fake_error), np.mean(G_error),np.mean(E_error),np.mean(new_measure),k_curr))
        log_txt.write('D_e : %.6f, D_real_e : %.6f, D_fake_e : %.6f, G_e : %.6f\n'%(np.mean(D_error),
            np.mean(D_real_error), np.mean(D_fake_error), np.mean(G_error)))
      
        r = sess.run([G_sample],feed_dict={z : test_z, isTrain : False})        
        mnist_4by4_save(np.reshape(r,(-1,64,64,1)),file_name + '/result_{}.png'.format(str(epoch).zfill(3)))
        
        r = sess.run([D_real],feed_dict={u : test_normal_data[0:16], isTrain : False})        
        mnist_4by4_save(np.reshape(r,(-1,64,64,1)),file_name + '/D_{}.png'.format(str(epoch).zfill(3)))
        
        r = sess.run([re_image],feed_dict={u : test_normal_data[0:16],isTrain : False})        
        mnist_4by4_save(np.reshape(r,(-1,64,64,1)),file_name + '/origin_{}.png'.format(str(epoch).zfill(3)))

        np.random.seed(int(time.time()))


        G_error = []
        D_error = []
        E_error = []
        D_fake_error = []     
        D_real_error = []
        new_measure = []

    log_txt.close()
    gan_loss_graph_save(G_loss = hist_G,D_loss=hist_D,path = file_name + '/loss_graph.png')   
    saver = tf.train.Saver()
    saver.save(sess,file_name + '/para.cktp')

    end = time.time()-start

    print("total time : ",end)

D_e : -83.492, D_real_e : 50.586, D_fake_e : 34.503, G_e : 35.169, E_e : 0.000, new_measure : 63.877, k_curr : 0.676337
D_e : 31.511, D_real_e : 37.095, D_fake_e : 25.650, G_e : 26.109, E_e : 0.000, new_measure : 39.166, k_curr : 0.278010
D_e : 28.121, D_real_e : 31.556, D_fake_e : 21.749, G_e : 22.128, E_e : 0.000, new_measure : 32.955, k_curr : 0.169794
D_e : 27.157, D_real_e : 27.663, D_fake_e : 19.403, G_e : 19.453, E_e : 0.000, new_measure : 29.087, k_curr : -0.079271
D_e : 23.396, D_real_e : 24.463, D_fake_e : 17.055, G_e : 17.116, E_e : 0.005, new_measure : 25.988, k_curr : -0.057276
D_e : 20.278, D_real_e : 21.047, D_fake_e : 14.672, G_e : 14.712, E_e : 0.018, new_measure : 22.123, k_curr : -0.000239
D_e : 18.069, D_real_e : 18.223, D_fake_e : 12.711, G_e : 12.708, E_e : 0.065, new_measure : 19.397, k_curr : 0.134522
D_e : 15.696, D_real_e : 16.151, D_fake_e : 11.298, G_e : 11.356, E_e : 0.238, new_measure : 17.170, k_curr : -0.005487
D_e : 14.459, D_real_e : 14.641, D_fake_e :

D_e : 4.314, D_real_e : 4.313, D_fake_e : 2.957, G_e : 3.013, E_e : 5.959, new_measure : 4.520, k_curr : 0.013776
D_e : 4.305, D_real_e : 4.307, D_fake_e : 2.955, G_e : 3.015, E_e : 5.971, new_measure : 4.525, k_curr : 0.013743
D_e : 4.290, D_real_e : 4.292, D_fake_e : 2.950, G_e : 3.006, E_e : 5.985, new_measure : 4.503, k_curr : 0.009063
D_e : 4.267, D_real_e : 4.269, D_fake_e : 2.936, G_e : 2.989, E_e : 6.006, new_measure : 4.443, k_curr : 0.004862
D_e : 4.262, D_real_e : 4.265, D_fake_e : 2.921, G_e : 2.984, E_e : 6.014, new_measure : 4.464, k_curr : 0.009472
D_e : 4.225, D_real_e : 4.230, D_fake_e : 2.905, G_e : 2.961, E_e : 6.034, new_measure : 4.415, k_curr : 0.009373
D_e : 4.240, D_real_e : 4.264, D_fake_e : 2.923, G_e : 2.990, E_e : 6.004, new_measure : 4.460, k_curr : -0.004076
D_e : 4.210, D_real_e : 4.211, D_fake_e : 2.886, G_e : 2.941, E_e : 6.068, new_measure : 4.407, k_curr : 0.015024
D_e : 4.177, D_real_e : 4.190, D_fake_e : 2.871, G_e : 2.939, E_e : 6.084, new_measure 

## network load

In [ ]:
if not os.path.isdir(file_name) :
    os.mkdir(file_name)

    
sess = tf.InteractiveSession()
    
new_saver = tf.train.import_meta_graph(file_name + '/para.cktp.meta')
new_saver.restore(sess, tf.train.latest_checkpoint(file_name + '/'))


z = sess.graph.get_tensor_by_name("z:0")
u = sess.graph.get_tensor_by_name("u:0")

isTrain = sess.graph.get_tensor_by_name("isTrain:0")

G_sample = sess.graph.get_tensor_by_name("G_sample:0")

D_real = sess.graph.get_tensor_by_name('D_real:0')                       # D(x)
D_fake = sess.graph.get_tensor_by_name('D_fake:0')         # D(G(z))


D_real_loss = sess.graph.get_tensor_by_name('D_real_loss:0')
D_fake_loss = sess.graph.get_tensor_by_name('D_fake_loss:0')

D_loss = sess.graph.get_tensor_by_name("D_loss:0")
G_loss = sess.graph.get_tensor_by_name("G_loss:0")


D_optim = sess.graph.get_operation_by_name("D_optim")
G_optim = sess.graph.get_operation_by_name("G_optim")



